In [ ]:
import numpy as np
import plotly.express as px
def plot_embedding(embedding, labels=None):
    """
    Plot a 2D embedding with optional labels for coloring.

    Args:
        embedding (ndarray): shape (n_samples, 2)
        labels (array-like, optional): labels for coloring points
    """
    if labels is None:
        labels = np.zeros(embedding.shape[0])  # default: all same

    fig = px.scatter(
        x=embedding[:, 0],
        y=embedding[:, 1],
        color=labels.astype(str),  # convert to string for discrete legend
        title="2D Embedding Visualization",
        labels={"x": "Dim 1", "y": "Dim 2", "color": "Label"}
    )

    fig.update_layout(
        width=1000,
        height=800,
        title="Custom size Plotly figure"
    )


    fig.show()

In [ ]:
f_path='/Users/chandrasekharmukherjee/Data/scRNA_IBD/'
d_path=['bcells_final.h5ad',
'cd4tcells_final.h5ad',
'cd8tcells_final.h5ad',
'epicolonic_final.h5ad',
'fibperi_final.h5ad',
'ilc_final.h5ad',
'ilealepi_final.h5ad',
'myeloid_final.h5ad',
'plasmacells_final.h5ad']

subset_number=1

import scanpy as sc
adata=sc.read_h5ad(f_path+d_path[subset_number])
X=adata.obsm['X_harmony']
n=X.shape[0]
print(n)
label=adata.obs['final_analysis'].to_numpy()
pt_label=adata.obs['Patient'].to_numpy()


In [ ]:
import scanpy as sc
nk_path='/Users/chandrasekharmukherjee/PycharmProjects/Personal/cplearn_biotest/NK_cells.h5ad'
adata = sc.read_h5ad(nk_path)
X=adata.obsm['X_HARMONY']
label=adata.obs['ident'].to_numpy()

In [ ]:
import data_call
import numpy as np
d_name='MNIST'
X,label=data_call.get_dataset(d_name, mode='PCA')

In [ ]:
import umap
import numpy as np
reducer=umap.UMAP()
X_umap=reducer.fit_transform(X)

#X_umap=np.array(adata.obsm['X_umap'])

In [ ]:
print(len(set(label)),X.shape)

In [ ]:
from cplearn_v3.corespect import Corespect
corespect = Corespect(X)

In [ ]:
corespect.extract_layers(method='stable_rank',layer_extraction_params={'q':20,'r':10,'core_fraction':0.2,'resolution':1.25})

In [ ]:
Layers_part=corespect.layers_
c=0
for layer in Layers_part:
    c+=len(layer)
    print(len(layer),len(set(layer)))
print(c,X.shape[0])

In [ ]:
for layer1 in Layers_part:
    for layer2 in Layers_part:
        print(len(set(layer1).intersection(set(layer2))),end=' ')

    print('\n')

In [ ]:
import numpy as np
n=X.shape[0]
r_label=np.empty(n, dtype=object)
for i,layer in enumerate(Layers_part):
    for node in layer:
        r_label[node]= str(i) #str(label[node])+ str(i)

plot_embedding(X_umap,labels=r_label)

In [ ]:
X_umap_core=reducer.fit_transform(X[Layers_part[0]])
plot_embedding(X_umap_core,label[Layers_part[0]])

In [ ]:
import leidenalg
import numpy as np
G=corespect.G
n=G.vcount()
G_core=G.induced_subgraph(Layers_part[0])
print(G_core.vcount())
partition = leidenalg.find_partition(
    G_core,
    leidenalg.RBConfigurationVertexPartition,  # modularity-based partition
    resolution_parameter=0.7, # <-- control resolution here
    seed = np.random.randint(int(1e9))
)

# Map back to original node ids
id_map = np.array(G_core.vs["id"], dtype=int)

new_labels = -1 * np.ones(n, dtype=int)
new_labels[id_map] = partition.membership

print(len(set(new_labels)))



In [ ]:
plot_embedding(X_umap,labels=label)


In [ ]:
from cplearn_v2.corespect import Corespect as CS_old
corespect_old = CS_old(X)
corespect_old.extract_layers(method='stable_rank',core_fraction=0.2,layer_extraction_params={'q':20,'r':20,'purify':1})
Layers=corespect_old.layers_
c=0
for layer in Layers:
    c+=len(layer)
    print(len(layer),len(set(layer)))
print(c,X.shape[0])

In [ ]:
r_label=-1*np.ones(len(label))
for i,layer in enumerate(Layers):
    for node in layer:
        r_label[node]=i

plot_embedding(X_umap,labels=r_label)

In [ ]:
from cplearn_v2.corespect import Corespect
corespect = Corespect(X)

In [ ]:
Total_Layers=[]
for purify in [1]:
    corespect.extract_layers(core_fraction=0.2,layer_extraction_params={'q':20,'r':10})
    Layers=corespect.layers_
    Total_Layers.append(Layers)
    c=0
    for layer in Layers:
        c+=len(layer)
        print(len(layer),len(set(layer)))
    print(c,X.shape[0])

    print('\n\n')

In [ ]:
import numpy as np
Layers_part1=Total_Layers[0]
n=X.shape[0]
r_label=np.empty(n, dtype=object)
for i,layer in enumerate(Layers_part1):
    for node in layer:
        r_label[node]=str(i)

plot_embedding(X_umap,labels=r_label)